## Imports

https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Linear_Programming.ipynb


Integer Optimization:
- https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Beyond_Linear_Programming.ipynb

In [1]:
from docplex.mp.model import Model
import numpy as np
import pandas as pd

In [2]:
# load data

players = pd.read_csv("../data/processed-data/0_selection_players.csv", index_col=0)
n_players = players.shape[0]
salaries = pd.read_csv("../data/processed-data/0_selection_salary.csv", index_col=0,).drop('Player', axis=1).values
n_time = salaries.shape[1]
n_trials = int(salaries.shape[0]/n_players)
salaries = salaries.reshape((n_trials, n_players, n_time))
market_values = salaries/0.1



# Set up all players
# use binary var cube
# (time, player, and simulation)


# set up goalkeeping separately
# set up DF, MF and Fw using separate dataset



In [3]:
# create model
ftcp = Model(name = "Rarita Football Team Composition")

max_age = 40
starting_budget = 15000000
rho = 0.05

time = np.arange(n_time)
trials = np.arange(n_trials)

In [4]:
# Decision variables
# assigned = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, keys3=trials, name = "assign_%s_%s_%s")
assigned = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "assign_%s_%s")

budget = ftcp.continuous_var_list(keys = time, name = "Budget_%s", lb=0)
team_value = ftcp.continuous_var_list(keys = time, name = "Team Value_%s")

sunset_value = ftcp.continuous_var(name = "Sunset Value_")
optim_value = ftcp.continuous_var(name = "Expectation of Team Value")

In [5]:
# Non decision variables
# Total number of players
N_total = ftcp.integer_var_list(keys = time, name = 'Total number of Players_%s', lb=22, ub=25)

# Number of players in each role
N_gk = ftcp.integer_var_list(keys = time, name = "Number of goalkeepers_%s", lb=2, ub=3)
N_df = ftcp.integer_var_list(keys = time, name = "Number of defenders_%s", lb=8, ub=10)
N_mf = ftcp.integer_var_list(keys = time, name = "Number of midfielders_%s", lb=8, ub=10)
N_fw = ftcp.integer_var_list(keys = time, name = "Number of forwards_%s", lb=3, ub=4)       

In [6]:
# Set up Constraints for number of players
for year in time:
    
    previous_budget = starting_budget if year == 0 else budget[year-1]   
    ftcp.add_constraint(budget[year] == previous_budget 
                        - ftcp.sum(assigned[index, year] * salaries[0, index, year] for index in range(n_players)))
    
    # Team value = Player values - Wages + Borrowing - Lending
    ftcp.add_constraint(team_value[year] == ftcp.sum(assigned[index, year] * market_values[0, index, year] for index in range(n_players))/((1 + rho)**year)
                        - ftcp.sum(assigned[index, year] * salaries[0, index, year] for index in range(n_players)))
    
    ftcp.add_constraint(N_gk[year] == ftcp.sum(assigned[index, year] * player['GK'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_df[year] == ftcp.sum(assigned[index, year] * player['DF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_mf[year] == ftcp.sum(assigned[index, year] * player['MF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_fw[year] == ftcp.sum(assigned[index, year] * player['FW'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_gk[year] + N_df[year] + N_mf[year] + N_fw[year] == N_total[year])

    for index, player in players.iterrows():
        ftcp.add_constraint(assigned[index, year] * (player['Age'] + year) <= max_age )
        
ftcp.add_constraint(sunset_value == ftcp.sum(assigned[index, time[-1]] * market_values[0, index, time[-1]] for index in range(n_players))/((1 + rho)**(n_time)))
ftcp.add_constraint(optim_value == ftcp.sum(team_value) + sunset_value)

docplex.mp.LinearConstraint[](Expectation of Team Value,EQ,Team Value_0+Team Value_1+Team Value_2+Team Value_3+Team Value_4+Team Value_5+Team Value_6+Team Value_7+Team Value_8+Team Value_9+Sunset Value_)

In [7]:
ftcp.maximize(optim_value)
ftcp.print_information()

Model: Rarita Football Team Composition
 - number of variables: 28292
   - binary=28220, integer=50, continuous=22
 - number of constraints: 28292
   - linear=28292
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [8]:
ftcp.solve(log_output=True, clean_before_solve=True)
ftcp.solve_status

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 3 times.
MIP Presolve eliminated 28232 rows and 20180 columns.
MIP Presolve added 1257 rows and 1257 columns.
MIP Presolve modified 249 coefficients.
Aggregator did 67 substitutions.
Reduced MIP has 1250 rows, 9302 columns, and 23532 nonzeros.
Reduced MIP has 8026 binaries, 1275 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.21 sec. (82.75 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1250 rows, 9302 columns, and 23532 nonzeros.
Reduced MIP has 8026 binaries, 1275 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (13.43 ticks)
Probing fixed 1472 vars, tightened 1613 bounds.
Probing time = 3.88 sec. (1499.09 ticks)
Clique table members: 9.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.04 sec. (11.68

<JobSolveStatus.INFEASIBLE_OR_UNBOUNDED_SOLUTION: 5>